# gBizINFO API データ品質検証

このノートブックでは、経済産業省が提供するgBizINFO APIの**データ品質**を検証します。
具体的に以下の点を明らかにします：

1. **時間的整合性**: 公開日と実際のデータ更新日の乖離
2. **網羅性**: 全省庁のデータが適切に統合されているか
3. **年次分布**: 各年のデータ量の偏り（2015-2025年）
4. **欠損値**: 重要フィールドの欠損率

## 前提条件
- APIキーを取得済み
- Python 3.9以上
- 必要ライブラリ: pandas, numpy, matplotlib, requests

## 分析者
Rioto3 (@Rioto3)

In [4]:
import os
from dotenv import load_dotenv

# .envファイルの読み込み
load_dotenv()

# APIトークンの設定（環境変数または直接指定）
API_TOKEN = os.getenv("GBIZINFO_API_TOKEN", "DTcLxzo1lZaUYaQPVdSRxdS4MzlXNCs4")  # 動作確認用トークン

# 基本URL
BASE_URL = "https://info.gbiz.go.jp/hojin"

# リクエストヘッダー
HEADERS = {
    'X-hojinInfo-api-token': API_TOKEN
}

In [ ]:
import requests
import pandas as pd
from datetime import datetime

def get_yearly_update_count(start_year=2015, end_year=datetime.now().year):
    results = {}
    
    for year in range(start_year, end_year + 1):
        from_date = f"{year}0101"
        to_date = f"{year}1231"
        
        # 各情報カテゴリごとにリクエスト
        categories = [
            'updateInfo', 
            'updateInfo/certification', 
            'updateInfo/commendation', 
            'updateInfo/finance', 
            'updateInfo/patent', 
            'updateInfo/procurement', 
            'updateInfo/subsidy', 
            'updateInfo/workplace'
        ]
        
        total_count = 0
        for category in categories:
            url = f"{BASE_URL}/v1/hojin/{category}"
            
            params = {
                'from': from_date,
                'to': to_date,
                'page': 1,
                'limit': 1
            }
            
            try:
                response = requests.get(
                    url, 
                    headers=HEADERS, 
                    params=params
                )
                
                if response.status_code == 200:
                    data = response.json()
                    total_count += int(data.get('totalCount', 0))
                
            except Exception as e:
                print(f"Error fetching data for {year} - {category}: {e}")
        
        results[year] = total_count
    
    return results

# 年次データ数を取得
yearly_counts = get_yearly_update_count()

# データフレームに変換して表示
df = pd.DataFrame.from_dict(yearly_counts, orient='index', columns=['更新件数'])
df.index.name = '年'
print(df)
